In [1]:
from raw_python.Bagging import create_models, create_bags, evaluate_stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import random
from raw_python.BaggingSA import BaggingSA
from raw_python.DatasetsHandle import get_dataset
from tabulate import tabulate

seed = 41
np.random.seed(seed)
random.seed(seed)

In [2]:
dataset_name = 'students_dropout'
k_cross = 4
reps = 3
params = {
    'T0': 2,
    'cooling_method': 'geometric',
    'alpha': 0.995,
    'max_iterations': 2000,
    'feature_mutation_chance': 0.25,
    'test_split_amount': 5,
    'beta': 0.75,
    'gamma': 0.25,
    'delta': 0.05,
    'epsilon': 0.0,
    'n_trees': 20,
}

In [3]:



def evaluate_bagging(X_train, y_train, X_test, y_test):
    n_trees = params['n_trees']
    bags = create_bags(X_train, y_train, bags_amount=n_trees)
    models = create_models(bags=bags)
    stats = evaluate_stats(X=X_test, y=y_test, models=models)
    return stats['accuracy'], stats['f1'], bags


    

def evaluate_bagging_sa(X_train, y_train, X_test, y_test, bags):
    T0 = params['T0']
    cooling_method = params['cooling_method']
    alpha = params['alpha']
    max_iterations = params['max_iterations']
    feature_mutation_chance = params['feature_mutation_chance']
    test_split_amount = params['test_split_amount']
    n_trees = params['n_trees']
    beta = params['beta']
    delta = params['delta']
    gamma = params['gamma']
    epsilon = params['epsilon']
    bagging_sa = BaggingSA(X=X_train, y=y_train,
                            T0=T0, cooling_method=cooling_method, alpha=alpha, max_iterations=max_iterations, n_trees=n_trees,
                            feature_mutation_chance=feature_mutation_chance, test_split_amount=test_split_amount, beta=beta, gamma=gamma, delta=delta, epsilon=epsilon)
    models, fitness = bagging_sa.run(X_for_test=X_test, y_for_test=y_test, monitor_fun=fun_monitor, get_fitness=True, initial_bags=bags)
    stats = evaluate_stats(X_test, y_test, models=models)
    return stats['accuracy'], stats['f1'], models, fitness

def fun_monitor(iteration, T, best_fitness, fitness, new_fitness, accuracy):
    if accuracy is None:
        accuracy = 0.0
    # print(f"    I: {iteration}, T: {T:.3f}, Best fitness: {best_fitness:.3f}, Fitness: {fitness:.3f}, New fitness: {new_fitness:.3f}, Accuracy: {accuracy:.3f}")

In [4]:
result = []
X,y = get_dataset(dataset_name)
random_indices = np.arange(X.shape[0])
np.random.shuffle(random_indices)
X = X[random_indices]
y = y[random_indices]

sub_groups_X = np.array_split(np.array(X), k_cross)
sub_groups_y = np.array_split(np.array(y), k_cross) 

for k in range(k_cross):
    X_train = np.concatenate(sub_groups_X[:k] + sub_groups_X[k+1:])
    y_train = np.concatenate(sub_groups_y[:k] + sub_groups_y[k+1:])
    X_test = sub_groups_X[k]
    y_test = sub_groups_y[k]
    for r in range(reps):
        acc_bagging, f1_bagging, bags = evaluate_bagging(X_train, y_train, X_test, y_test)
        acc_sa, f1_sa, models, fitness = evaluate_bagging_sa(X_train, y_train, X_test, y_test, bags)
        
        result.append([k+1, r+1, fitness, acc_sa, acc_bagging, f1_sa, f1_bagging])
        print(f"Fold {k+1:2}/{k_cross:2} | Rep {r+1:2}/{reps:2} | Fitness: {fitness:.3f} | SA: {acc_sa:.3f} | Bagging: {acc_bagging:.3f} | F1 SA: {f1_sa:.3f} | F1 Bagging: {f1_bagging:.3f}")
        df = pd.DataFrame(result, columns=['Fold', 'Rep', 'Fitness', 'SA', 'Bagging', 'F1 SA', 'F1 Bagging'])
        df.to_csv(f'./../res/test_bagging.csv', index=False)

Fold  1/ 4 | Rep  1/ 3 | Fitness: 0.800 | SA: 0.754 | Bagging: 0.751 | F1 SA: 0.666 | F1 Bagging: 0.680


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv(f'./../res/test_bagging.csv')

tmp = df.copy().round(3)
print(tabulate(tmp, headers='keys', tablefmt='pretty', showindex=False))

acc_mean = df['SA'].mean()
bagging_mean = df['Bagging'].mean()

sa_f1 = df['F1 SA'].mean()
bagging_f1 = df['F1 Bagging'].mean()

print(f"Accuracy mean: {acc_mean:.3f}")
print(f"Bagging mean:  {bagging_mean:.3f}")
print("=====================================")
print(f"SA F1 mean: {sa_f1:.3f}")
print(f"Bagging F1 mean: {bagging_f1:.3f}")

+------+-----+---------+-------+---------+-------+------------+
| Fold | Rep | Fitness |  SA   | Bagging | F1 SA | F1 Bagging |
+------+-----+---------+-------+---------+-------+------------+
| 1.0  | 1.0 |  0.794  | 0.759 |  0.753  | 0.675 |   0.684    |
| 1.0  | 2.0 |  0.788  | 0.762 |  0.744  | 0.691 |   0.666    |
| 1.0  | 3.0 |  0.786  | 0.742 |  0.757  | 0.656 |   0.678    |
| 2.0  | 1.0 |  0.794  | 0.734 |  0.751  | 0.641 |   0.683    |
| 2.0  | 2.0 |  0.791  | 0.742 |  0.75   | 0.67  |   0.685    |
| 2.0  | 3.0 |  0.797  | 0.739 |  0.726  | 0.662 |   0.642    |
+------+-----+---------+-------+---------+-------+------------+
Accuracy mean: 0.746
Bagging mean:  0.747
SA F1 mean: 0.666
Bagging F1 mean: 0.673


    +------+-----+---------+----------+---------+---------------+
    | Fold | Rep | Fitness | Accuracy | Bagging | BaggingRandom |
    +------+-----+---------+----------+---------+---------------+
    | 1.0  | 1.0 |  0.822  |  0.797   |  0.793  |     0.817     |
    | 1.0  | 2.0 |  0.828  |  0.793   |  0.807  |     0.823     |
    | 1.0  | 3.0 |  0.847  |  0.783   |  0.83   |     0.813     |
    | 2.0  | 1.0 |  0.803  |  0.839   |  0.849  |     0.876     |
    | 2.0  | 2.0 |  0.822  |  0.853   |  0.883  |     0.873     |
    | 2.0  | 3.0 |  0.795  |  0.873   |  0.89   |     0.89      |
    | 3.0  | 1.0 |  0.828  |  0.866   |  0.839  |     0.846     |
    | 3.0  | 2.0 |  0.824  |  0.806   |  0.803  |     0.839     |
    | 3.0  | 3.0 |  0.86   |  0.856   |  0.846  |     0.803     |
    +------+-----+---------+----------+---------+---------------+
    Fitness mean:  0.826
    Accuracy mean: 0.830
    Bagging mean:  0.838
    Bagging Random mean:  0.842

In [ ]:
#statistic tests
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu, shapiro

t1_start, p1 = shapiro(df['Bagging'])
t2_start, p2 = shapiro(df['Accuracy'])

print(f"Shapiro Bagging: t-statistic = {t1_start:.3f}, p-value = {p1:.3f}")
print(f"Shapiro BaggingSA: t-statistic = {t2_start:.3f}, p-value = {p2:.3f}")

if p1 > 0.05 and p2 > 0.05:
    t_stat, p_value = ttest_rel(df['Accuracy'], df['Bagging'], alternative='greater')
    print(f"t-test: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")
    txt = 'BaggingSA is greater than Bagging' if p_value < 0.05 else 'BaggingSA is not greater than Bagging'
    print(txt)
else:
    w_stat, p_value = wilcoxon(df['Accuracy'], df['Bagging'], alternative='greater')
    print(f"Wilcoxon: w-statistic = {w_stat:.3f}, p-value = {p_value:.3f}")
    txt = 'BaggingSA is greater than Bagging' if p_value < 0.05 else 'BaggingSA is not greater than Bagging'
    print(txt)

KeyError: 'Accuracy'